# Use this python notebook to analyze, understand, and interpret the results of your work in lab 1.<br>
1. First of all, we import the needed Python libraries, read the file with the densities of water as a function of temperature, _ro.csv_ , and enter the working temperature. The density of water is linearly interpolated from the values in _ro.csv_ <br>
__Q. Show the instructor how the linear interpolation is carried out__

In [ ]:
#lab 1
import numpy as np
import pandas as pd
from matplotlib import pyplot
from scipy.optimize import curve_fit

def polynomial(x,a,b,c):
    return a + b * x + c * x * x

def lin_interp(i,dec):
    ii = int(i)
    return ro[ii] + (ro[ii+1] - ro[ii]) * dec

def Msol(x,MA,MB):
    return (1 - x) * MA + x * MB
    
def xB_a(m,mB,MA,MB):
    return mB / MB / (mB / MB + (m - mB) / MA)
    
#reading the ro vs temp file
ro = pd.read_csv("ro.csv",sep=',')
ro = ro['density'].values

#working temp
t_work = float(input("Working temperature of your experiment (degrees Celsius) = "))
i,dec = divmod(t_work,1)
#interpolated temperature is temp
density = lin_interp(i,dec)
print('Density of water (g/ml) at the working temperature = ', density)

2. We now proceed with the calibration of the pycnometer

In [ ]:
n = int(input("Number of measurements = "))
mass = 0
for i in range(n):
    mass += float(input("Mass in this measurement = "))
#Average mass of water in the calibration 
mass = mass / n
#Volume of pyctnometer
Vpyc = mass / density
print('\nVolume of the pycnometer (ml) = ',Vpyc)
#Mass 1 =  11.396
#Mass 2 =  11.400
#Mass 3 =  11.399

3. Now we determine the approximate masses of each compound ($m_{A}'$ and $m_{B}'$) for a total approximate mass of the mixture $m_{mix}'=20$ g and 0.05 intervals for $x_{B}'$

In [ ]:
mtotp = 20 # take a total mass of 20 g
MB = float(input('Molar mass (g/mol) of component B (the organic compound) = '))
MA = 18.01528 # molar mass of water 
#n = int(input("Number of solutions to be prepared = "))
n = 21
print("\nApproximate masses you need to weigh for each component of these solutions")
xBp = np.linspace(0,1,n,endpoint=True)
xAp = 1 - xBp  
mBp = mtotp * xBp * MB / Msol(xBp,MA,MB)
mAp = mtotp - mBp
print('    xB\' mA(g)\' mB(g)\'')
for i in range(n):
    print('%7.2f%7.3f%7.3f' % (xBp[i],mAp[i],mBp[i]))

4. The weighted masses for each component $m_{A}$ and $m_{B}$ (they may vary from the above values with the prime), as well as the mass of each mixture (solution) inside the pycnometer $m_{mix,pyc}$ must be included in the excel file _masses.csv_ under the columns mA, mB and mpyc, respectively. From theses masses, $x_{B}$, the density of each mixture $d$ and its molar volume $V_{m}$ will be determined. Additionally, $V_{m}$ will be plotted as a function of $x_{B}$.<br>
__Q. Show the instructor how $V_{m}$ is determined__

In [ ]:
m = pd.read_csv("masses.csv",sep=',')
mB = m['mB']
mA = m['mA']
mt = mA + mB
mpyc  = m['mpyc']

xB  =  xB_a(mt.values,mB.values,MA,MB)
d   = mpyc.values / Vpyc
Vm  = Msol(xB,MA,MB) / d

print('        xB   d(g/ml) Vm(ml/mol)')
for i in range(len(d)):    
    print('%10.4f%10.4f%11.4f' % (xB[i],d[i],Vm[i]))

5. We now fit $V_{m}$ to a quadratic polynomial:<br>
$V_{m}=a+bx+cx^{2}$<br>
__Q. What are the units of $a$, $b$ and $c$__<br>
__Q. What is the RMS value given below?. What are the units of RMS?__

In [ ]:
#fit
best_vals, covar = curve_fit(polynomial, xB, Vm)
rms = 0
for i, value in enumerate(xB):
    rms += (polynomial(value,*best_vals) - Vm[i]) ** 2
rms = np.sqrt ( rms / len(xB) )
xx = np.linspace(0.,1.,num = 120)
Vm_fit = polynomial(xx,*best_vals)
print('\nQuadratic polynomial that best fits our data:\n')
print(best_vals[0],'+',best_vals[1],'* x +',best_vals[2],'* x * x')
print('\nRMS of the fit = ',rms)

pyplot.xticks(np.arange(0, 1.1, step=0.1))
pyplot.xlim(-0.02,1.02)
pyplot.plot(xB,Vm,'o',color='k',label='Data',mfc='none')
pyplot.plot(xx,Vm_fit,'-',color='k',label='Fit')
pyplot.legend()
pyplot.ylabel("Vm (ml/mol)")
pyplot.xlabel("xB")

6. Plotting now the partial molar volumes $\bar{V}_{A}$ and $\bar{V}_{B}$ and the change in molar volume on mixing $\Delta V_{m,mix}=V_{m}-V_{m}^{*}$.<br>
__Q. Discuss the shape of the plots with the instructor__

In [ ]:
slope = best_vals[1] + 2 * best_vals[2] * xx
VA_star = polynomial(0,*best_vals)
VB_star = polynomial(1,*best_vals)
Vm_star = VA_star * (1 - xx) + VB_star * xx
deltaV = Vm_fit - Vm_star
VA = Vm_fit - slope * xx
VB = VA + slope
pyplot.xticks(np.arange(0, 1.1, step=0.1))
pyplot.xlim(-0.02,1.02)
pyplot.plot(xx,VA,'-',color='b',label='VA')
pyplot.plot(xx,VB,'-',color='r',label='VB')
pyplot.legend()
pyplot.ylabel("Partial molar volumes (ml/mol)")
pyplot.xlabel("xB")
pyplot.show()

pyplot.xticks(np.arange(0, 1.1, step=0.1))
pyplot.xlim(-0.02,1.02)
pyplot.plot(xx,deltaV,'-',color='k',label='deltaV_m')
pyplot.legend()
pyplot.ylabel("DeltaVm (ml/mol)")
pyplot.xlabel("xB")
pyplot.show()

7. Write your code below to obtain $\bar{V}_{A}$, $\bar{V}_{B}$ and $\Delta V_{m,mix}$ only for the $x_{B}$ values of your mixtures

In [ ]:
'''
Hint: copy part of the code above and replace xx (which is an array with 120 values beetween 0 and 1) with your
actual xB values (those measured in the lab) 
'''
Vm_fit = polynomial(xB,*best_vals)
Vm_star = VA_star * (1 - xB) + VB_star * xB
deltaV = Vm_fit - Vm_star
slope = best_vals[1] + 2 * best_vals[2] * xB
VA = Vm_fit - slope * xB
VB = VA + slope
print(xB,VA)
print(xB,VB)
print(xB,deltaV)

8. Now, you have to complete the report and hand it over to the instructor before leaving the lab